# A small agent using tools 

In [4]:
!pip install tavily-python

  Using cached tiktoken-0.12.0-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
Using cached tiktoken-0.12.0-cp313-cp313-win_amd64.whl (879 kB)

   ---------------------------------------- 2/2 [tavily-python]



In [1]:
import openai
from openai import OpenAI
import os

In [2]:
# Set API key and base URL globally
api_key = os.getenv("OPENAI_API_KEY") # Use correct env var name from your .env or manually set it
print("OpenAI API Key:", api_key[:10] + "..." if api_key else "Not found")  # Only show first 10 chars for security

base_url = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")  # Use correct env var name from your .env
print("OpenAI Base URL:", base_url)

model = os.getenv("OPENAI_MODEL", "gpt-5-nano")  # Use correct env var name from your .env
print("OpenAI Model:", model)  # Print the model being used

# Initialize OpenAI client with API key and base URL
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)


OpenAI API Key: sk-d7GDhmo...
OpenAI Base URL: https://xiaoai.plus/v1
OpenAI Model: gpt-5-nano


In [3]:
message = "What is the capital of France?"

messages = "You are a helpful assistant. Please answer the question: " + message 
#this is the final prompt sent

response = client.chat.completions.create(
    model= model, # or "gpt-5-nano" for the preview version
    # temperature and max_tokens are not supported for gpt-5-nano
    # temperature=0.7, # controls randomness in the response (not supported for gpt-5-nano)
    # max_tokens=100, # maximum number of tokens in the response (not supported for gpt-5-nano)
    top_p=1.0,
    messages=[
        {"role": "user", "content": message}
    ]
)

print(response.choices[0].message.content)
print("Model used is:", model)

Paris.
Model used is: gpt-5-nano


In [7]:
AGENT_SYSTEM_PROMPT = """
你是一个智能问答助手。你的任务是分析用户的请求，并使用可用工具一步步地解决问题。

# 可用工具:
- `search(query: str) -> str` : 该工具允许你在互联网上搜索信息。传入一个查询字符串，返回搜索结果的摘要。


# 行动格式:
你的回答必须严格遵循以下格式。首先是你的思考过程，然后是你要执行的具体行动，每次回复只输出一对Thought-Action：
Thought: [这里是你的思考过程和下一步计划]
Action: [这里是你要调用的工具，格式为 function_name(arg_name="arg_value")]

# 任务完成:
当你收集到足够的信息，能够回答用户的最终问题时，你必须在`Action:`字段后使用 `finish(answer="...")` 来输出最终答案。

请开始吧！
"""


Google API Key: AIzaSyAhvI...
Google Search Engine ID: 727efebf2d...


In [27]:
#test tool
search_results = search("What is the capital of France?")
print(search_results)

{'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - What is the capital of France?', 'totalResults': '648000000', 'searchTerms': 'What is the capital of France?', 'count': 10, 'startIndex': 1, 'inputEncoding': 'utf8', 'outputEncoding': 'utf8', 'safe': '

In [8]:
# 将所有工具函数放入一个字典，方便后续调用
available_tools = {
    "search": search,
}

In [ ]:
# define 